In [1]:
from functions_for_all_nutrients import *
from iron_hemoglobin_functions import *
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
username = !whoami
username

['alibow']

In [3]:
index_cols=['location_id','sex_id','age_group_id']

age_group_ids = list(range(7,16))
sex_ids = [2]
coverage_levels = [0.2,0.5,0.8]
years = [2021,2022,2023,2024,2025]

In [4]:
coverage_data_dir = f'/ihme/homes/{username[0]}/notebooks/vivarium_research_lsff/data_prep/outputs/lsff_input_coverage_data.csv'

In [5]:
location_ids = (list(
                pd.read_csv(coverage_data_dir)
                .location_id
                .unique()))
location_ids

[179,
 163,
 214,
 161,
 165,
 190,
 189,
 196,
 201,
 180,
 15,
 164,
 20,
 202,
 205,
 171,
 11,
 184,
 168,
 6,
 141,
 207,
 181,
 213,
 522]

In [6]:
nutrient = 'iron'
vehicles = ['wheat flour', 'maize flour']
effective_fractions = [1] * len(age_group_ids)

In [ ]:
baseline_coverage, counterfactual_coverage = get_baseline_and_counterfactual_coverage(coverage_data_dir,
                                             location_ids,
                                             nutrient,
                                             vehicles,
                                             years,
                                             coverage_levels, 'WRA')
counterfactual_coverage.head()

# HEMOGLOBIN

In [ ]:
from scipy.stats import norm

# mean and 0.975-quantile of normal distribution for mean difference (MD)
mean = 3
q_975 = 6.1

# 0.975-quantile of standard normal distribution (=1.96, approximately)
q_975_stdnorm = norm().ppf(0.975)

std = (q_975 - mean) / q_975_stdnorm # std dev of normal distribution

# Frozen normal distribution for MD, representing uncertainty in our effect size
hb_md_distribution = norm(mean, std)

In [ ]:
mean_difference_hemoglobin_fort = generate_normal_rr_deficiency_nofort_draws(mean, std, location_ids)
mean_difference_hemoglobin_fort.head()

In [ ]:
# NOTE:
# This approach currently does not consider that new coverage
# applies to those who are slightly more anemic
# than the overall population due to their lack
# of access to fortification

effective_baseline_coverage = get_effective_iron_hemoglobin_coverage(baseline_coverage.set_index(['location_id','vehicle','year']), 
                                                                     sex_ids, 
                                                                     age_group_ids, 
                                                                     effective_fractions, 
                                                                     years)
effective_counterfactual_coverage = get_effective_iron_hemoglobin_coverage(counterfactual_coverage.set_index(['location_id','vehicle','year','coverage_level']), 
                                                                     sex_ids, 
                                                                     age_group_ids, 
                                                                     effective_fractions, 
                                                                     years)

delta_effective_coverage = effective_counterfactual_coverage - effective_baseline_coverage
delta_effective_coverage

In [ ]:
mean_hgb = generate_hemoglobin_values(delta_effective_coverage, 
                                      mean_difference_hemoglobin_fort,
                                      location_ids, age_group_ids, sex_ids)
mean_hgb = mean_hgb.reset_index()
mean_hgb = (mean_hgb
            .loc[mean_hgb.year_id.isin([2025, np.nan])]
            .set_index([c for c in mean_hgb.columns if 'draw' not in c]))

mean_hgb.to_csv('anemia_files_wra/mean_hgb_wra.csv')
mean_hgb = (mean_hgb.reset_index()
            .rename(columns={'year_id':'year'})
            .set_index(['location_id','vehicle','sex_id','age_group_id','coverage_level','year']))

# note, expect to see missing results for locations excluded due to illogical values

mean_hgb

In [ ]:
hgb_sd = get_draws('modelable_entity_id',
                10488,
                source='epi',
                location_id=location_ids,
                age_group_id=age_group_ids,
                sex_id=sex_ids,
                year_id=2019,
                gbd_round_id=6,
                decomp_step='step4',
                status='best')
hgb_sd = (hgb_sd.set_index(['location_id','sex_id','age_group_id'])
 .filter([c for c in hgb_sd if 'draw' in c])
 .reset_index())
hgb_sd.to_csv('anemia_files_wra/sd_hgb_wra.csv')
hgb_sd.head()

In [ ]:
# get age-specific fertility rate
get_covariate_estimates(13, 
                               location_id=location_ids,
                               age_group_id=age_group_ids,
                               sex_id=sex_ids,
                               year_id=2019,
                               gbd_round_id=6,
                               decomp_step='step4').to_csv('anemia_files_wra/asfr.csv')


# get still birth to live birth ratio (not age-specific)
still = get_covariate_estimates(2267, 
                                location_id=location_ids,
                                year_id=2019,
                                gbd_round_id=6,
                                decomp_step='step4').to_csv('anemia_files_wra/still.csv')

In [11]:
# NOW RUN HEMOGLOBIN_TO_ANEMIA.R FILE BEFORE RUNNING NEXT CELL
assert "Did you run the R file?" == "Double check :)", "Make sure you run the R file!"

AssertionError: Make sure you run the R file!

In [7]:
anemia = load_anemia_prev_and_calculate_ylds('anemia_files_wra/anemia_prev_wra.csv')
baseline_anemia = (duplicate_over_simulation_years(anemia.loc[anemia.coverage_level=='baseline'], years)
                   .drop(columns='coverage_level')
                   .set_index(['location_id','age_group_id','sex_id','year','draw','vehicle']))
counterfactual_anemia = (anemia.loc[anemia.coverage_level != 'baseline']
                         .set_index(['location_id','age_group_id','sex_id','year','draw','vehicle','coverage_level']))
averted_anemia = baseline_anemia - counterfactual_anemia
averted_anemia

mild  \
location_id age_group_id sex_id year draw     vehicle     coverage_level             
6           7            2      2021 draw_0   maize flour NaN                  NaN   
                                              wheat flour NaN                  NaN   
                                     draw_1   maize flour NaN                  NaN   
                                              wheat flour NaN                  NaN   
                                     draw_10  maize flour NaN                  NaN   
...                                                                            ...   
522         15           2      2025 draw_999 maize flour 0.5             0.001062   
                                                          0.8             0.001701   
                                              wheat flour 0.2             0.000658   
                                                          0.5             0.001648   
                                                          0.8             0.002640   

                                                                          moderate  \
location_id age_group_id sex_id year draw     vehicle     coverage_level             
6           7            2      2021 draw_0   maize flour NaN                  NaN   
                                              wheat flour NaN                  NaN   
                                     draw_1   maize flour NaN                  NaN   
                                              wheat flour NaN                  NaN   
                                     draw_10  maize flour NaN                  NaN   
...                                                                            ...   
522         15           2      2025 draw_999 maize flour 0.5             0.002083   
                                                          0.8             0.003324   
                                              wheat flour 0.2             0.001293   
                                                          0.5             0.003221   
                                                          0.8             0.005133   

                                                                            severe  \
location_id age_group_id sex_id year draw     vehicle     coverage_level             
6           7            2      2021 draw_0   maize flour NaN                  NaN   
                                              wheat flour NaN                  NaN   
                                     draw_1   maize flour NaN                  NaN   
                                              wheat flour NaN                  NaN   
                                     draw_10  maize flour NaN                  NaN   
...                                                                            ...   
522         15           2      2025 draw_999 maize flour 0.5             0.000240   
                                                          0.8             0.000381   
                                              wheat flour 0.2             0.000149   
                                                          0.5             0.000370   
                                                          0.8             0.000587   

                                                                            anemic  \
location_id age_group_id sex_id year draw     vehicle     coverage_level             
6           7            2      2021 draw_0   maize flour NaN                  NaN   
                                              wheat flour NaN                  NaN   
                                     draw_1   maize flour NaN                  NaN   
                                              wheat flour NaN                  NaN   
                                     draw_10  maize flour NaN                  NaN   
...                                                                            ...   
522         15           2      2

In [34]:
anemia_cols = ['mild','moderate','severe','anemic','moderate_ylds','mild_ylds','severe_ylds','anemic_ylds']

anemia_diff = anemia.drop(columns='year').dropna()
pop = get_population(location_id=location_ids,
                    sex_id=sex_ids,
                    age_group_id=age_group_ids,
                    year_id=2019,
                    gbd_round_id=6,
                    decomp_step='step4')
anemia_diff = anemia_diff.merge(pop, on=['location_id','age_group_id','sex_id'])
for col in anemia_cols:
    anemia_diff[f'{col}'] = anemia_diff[f'{col}'] * anemia_diff['population']
anemia_diff = anemia_diff.groupby(['location_id','draw','vehicle','coverage_level']).sum().filter(anemia_cols)#.reset_index()
anemia_diff = anemia_diff.stack().reset_index().rename(columns={'level_4':'measure',0:'value'})
anemia_diff = pd.pivot_table(anemia_diff, index=['location_id','vehicle','coverage_level','measure'],
                            columns='draw',
                            values='value')

anemia_diff.to_pickle('results_raw/iron_anemia_diff_wra.pkl')
anemia_diff.head()

draw                                                      draw_0  \
location_id vehicle     coverage_level measure                     
6           maize flour baseline       anemic       4.476115e+07   
                                       anemic_ylds  8.991964e+05   
                                       mild         3.097320e+07   
                                       mild_ylds    1.238928e+05   
                                       moderate     1.318660e+07   

draw                                                      draw_1  \
location_id vehicle     coverage_level measure                     
6           maize flour baseline       anemic       4.648963e+07   
                                       anemic_ylds  9.704878e+05   
                                       mild         3.156428e+07   
                                       mild_ylds    1.262571e+05   
                                       moderate     1.422316e+07   

draw                                                     draw_10  \
location_id vehicle     coverage_level measure                     
6           maize flour baseline       anemic       4.112175e+07   
                                       anemic_ylds  8.109839e+05   
                                       mild         2.870835e+07   
                                       mild_ylds    1.148334e+05   
                                       moderate     1.189120e+07   

draw                                                    draw_100  \
location_id vehicle     coverage_level measure                     
6           maize flour baseline       anemic       4.593680e+07   
                                       anemic_ylds  9.341044e+05   
                                       mild         3.159908e+07   
                                       mild_ylds    1.263963e+05   
                                       moderate     1.369704e+07   

draw                                                    draw_101  \
location_id vehicle     coverage_level measure                     
6           maize flour baseline       anemic       4.812083e+07   
                                       anemic_ylds  9.875354e+05   
                                       mild         3.292830e+07   
                                       mild_ylds    1.317132e+05   
                                       moderate     1.451406e+07   

draw                                                    draw_102  \
location_id vehicle     coverage_level measure                     
6           maize flour baseline       anemic       4.568979e+07   
                                       anemic_ylds  9.390484e+05   
                                       mild         3.126304e+07   
                                       mild_ylds    1.250522e+05   
                                       moderate     1.376895e+07   

draw                                                    draw_103  \
location_id vehicle     coverage_level measure                     
6           maize flour baseline       anemic       4.246269e+07   
                                       anemic_ylds  8.185445e+05   
                                       mild         2.993098e+07   
                                       mild_ylds    1.197239e+05   
                                       moderate     1.204541e+07   

draw                                                    draw_104  \
location_id vehicle     coverage_level measure                     
6           maize flour baseline       anemic       5.046263e+07   
                                       anemic_ylds  1.114253e+06   
                                       mild         3.326637e+07   
                                       mild_ylds    1.330655e+05   
                                       moderate     1.629954e+07   

draw                                                    draw_105  \
location_id vehicle     coverage_level measure                     
6           maize flour baseline       anemic   

In [35]:
counts, rates = population_weight_values(averted_anemia.reset_index(), age_group_ids, sex_ids, location_ids)

In [36]:
counts = summarize_data(counts)
rates = summarize_data(rates)

In [37]:
anemia_counts_averted = (counts.loc[counts.severity=='anemic']
     .loc[counts.measure=='ylds']
     .set_index(['location_id','vehicle','coverage_level','year'])
     .drop(columns=['severity','measure']))
anemia_counts_averted.head()

draw                                                  draw_0        draw_1  \
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0   42072.254901  15003.730025   
                        0.5            2025.0  101453.252849  37072.841717   
                        0.8            2025.0  156619.366057  58627.890104   
15          wheat flour 0.2            2025.0     720.598941    311.929224   
                        0.5            2025.0    1798.227168    779.129147   

draw                                                draw_10      draw_100  \
location_id vehicle     coverage_level year                                 
6           wheat flour 0.2            2025.0  16446.937406  14602.881792   
                        0.5            2025.0  40476.960098  36076.264330   
                        0.8            2025.0  63758.349019  57042.164593   
15          wheat flour 0.2            2025.0    271.624345    236.229893   
                        0.5            2025.0    678.494583    590.220422   

draw                                                draw_101       draw_102  \
location_id vehicle     coverage_level year                                   
6           wheat flour 0.2            2025.0   46832.360927   39306.310932   
                        0.5            2025.0  112941.947405   95158.307233   
                        0.8            2025.0  174359.827887  147473.197927   
15          wheat flour 0.2            2025.0     530.530847     442.299084   
                        0.5            2025.0    1324.419975    1104.519302   

draw                                               draw_103       draw_104  \
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0   7059.133373   49490.169742   
                        0.5            2025.0  17531.681828  119658.112515   
                        0.8            2025.0  27866.320750  185192.370733   
15          wheat flour 0.2            2025.0     97.620852     504.216102   
                        0.5            2025.0    243.982413    1258.894234   

draw                                               draw_105       draw_106  \
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0  15348.610589   38627.835985   
                        0.5            2025.0  38002.409944   93003.752575   
                        0.8            2025.0  60219.920265  143359.551019   
15          wheat flour 0.2            2025.0    157.825212     618.972219   
                        0.5            2025.0    394.393016    1544.528480   

draw                                           ...      draw_990  \
location_id vehicle     coverage_level year    ...                 
6           wheat flour 0.2            2025.0  ...  25335.107706   
                        0.5            2025.0  ...  62191.972000   
                        0.8            2025.0  ...  97713.063786   
15          wheat flour 0.2            2025.0  ...    331.198298   
                        0.5            2025.0  ...    827.239436   

draw                                               draw_991       draw_992  \
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0  25001.965362   70987.185129   
                        0.5            2025.0  61199.351872  170401.599541   
                        0.8            2025.0  95882.543371  261854.073719   
15          wheat flour 0.2            2025.0    311.810946     552.024467   
                        0.5            2025.0    778.872322    1378.069340   

draw                                                draw_993      draw_994  \
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0   57434.134346  12635.850083   
       

In [38]:
anemia_rates_averted = (rates.loc[rates.severity=='anemic']
     .loc[rates.measure=='ylds']
     .set_index(['location_id','vehicle','coverage_level','year'])
     .drop(columns=['severity','measure']))
anemia_rates_averted.head()

draw                                              draw_0     draw_1  \
location_id vehicle     coverage_level year                           
6           wheat flour 0.2            2025.0   9.439721   3.366376   
                        0.5            2025.0  22.762993   8.318007   
                        0.8            2025.0  35.140574  13.154297   
15          wheat flour 0.2            2025.0   3.791763   1.641359   
                        0.5            2025.0   9.462200   4.099747   

draw                                             draw_10   draw_100  \
location_id vehicle     coverage_level year                           
6           wheat flour 0.2            2025.0   3.690187   3.276438   
                        0.5            2025.0   9.081786   8.094405   
                        0.8            2025.0  14.305415  12.798509   
15          wheat flour 0.2            2025.0   1.429277   1.243032   
                        0.5            2025.0   3.570212   3.105716   

draw                                            draw_101   draw_102  draw_103  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0  10.507743   8.819129  1.583853   
                        0.5            2025.0  25.340703  21.350601  3.933571   
                        0.8            2025.0  39.120989  33.088455  6.252346   
15          wheat flour 0.2            2025.0   2.791632   2.327360  0.513677   
                        0.5            2025.0   6.969045   5.811937  1.283826   

draw                                            draw_104   draw_105  \
location_id vehicle     coverage_level year                           
6           wheat flour 0.2            2025.0  11.104074   3.443757   
                        0.5            2025.0  26.847604   8.526573   
                        0.8            2025.0  41.551479  13.511500   
15          wheat flour 0.2            2025.0   2.653165   0.830470   
                        0.5            2025.0   6.624251   2.075280   

draw                                            draw_106  ...   draw_990  \
location_id vehicle     coverage_level year               ...              
6           wheat flour 0.2            2025.0   8.666900  ...   5.684420   
                        0.5            2025.0  20.867185  ...  13.953968   
                        0.8            2025.0  32.165479  ...  21.923810   
15          wheat flour 0.2            2025.0   3.257007  ...   1.742752   
                        0.5            2025.0   8.127247  ...   4.352901   

draw                                            draw_991   draw_992  \
location_id vehicle     coverage_level year                           
6           wheat flour 0.2            2025.0   5.609673  15.927344   
                        0.5            2025.0  13.731254  38.232884   
                        0.8            2025.0  21.513097  58.752010   
15          wheat flour 0.2            2025.0   1.640737   2.904731   
                        0.5            2025.0   4.098395   7.251346   

draw                                            draw_993   draw_994  \
location_id vehicle     coverage_level year                           
6           wheat flour 0.2            2025.0  12.886455   2.835097   
                        0.5            2025.0  30.894212   7.022695   
                        0.8            2025.0  47.418951  11.133344   
15          wheat flour 0.2            2025.0   3.858744   1.416230   
                        0.5            2025.0   9.625328   3.537916   

draw                                            draw_995   draw_996  \
location_id vehicle     coverage_level year                           
6           wheat flour 0.2            2025.0   6.298770  11.878623   
                        0.5            2025.0  15.340628  28.498907   
                        0.8            2025.0  23.915798  43.775576   
15          wheat flour 0.2            2025.0   1.9633

In [8]:
# total iron responsive anemia burden
iron_responsive_anemia_sequelae = [1004, 1005, 1006, 1008, 1009, 1010, 1012, 1013, 
                                   1014, 1016, 1017, 1018, 1020, 1021, 1022, 1024, 1025, 1026, 
                                   1028, 1029, 1030, 1032, 1033, 1034, 1361, 1364, 1367, 1373, 1376, 
                                   1379, 1385, 1388, 1391, 1397, 1400, 1403, 1409, 1412, 1415, 1421, 
                                   1424, 1427, 1433, 1436, 1439, 1445, 1448, 1451, 5213, 5216, 5219, 
                                   5222, 5225, 5228, 5237, 5240, 5243, 5246, 5249, 5252, 5261, 5264, 
                                   5267, 5270, 5273, 5276, 4985, 4988, 4991, 4994, 4997, 5000, 5009, 
                                   5012, 5015, 5678, 5681, 5684, 7214, 7217, 7220, 4952, 4955, 4958, 
                                   4961, 4964, 4967, 4976, 4979, 4982, 5627, 5630, 5633, 7202, 7205, 
                                   7208, 5393, 5396, 5399, 182, 183, 184, 240, 241, 242, 177, 178, 
                                   179, 144,145,146,172,173,174,525,526,527,1106,1107,1108,537,538,
                                   539,206,207,208, 22989, 22990, 22991, 22992, 22993, 22999, 23000, 
                                   23001, 23002, 23003, 23009, 23010, 23011, 23012, 23013,
                                   5567, 5570, 5573, 5579, 5582, 5585,
                                   23030, 23031, 23032, 23034, 23035, 23036, 23038, 23039, 23040,
                                   23042, 23043, 23044, 23046, 23047, 23048]

ira_ylds = get_draws('sequela_id', iron_responsive_anemia_sequelae, 
                 source='como',
                 location_id=location_ids, 
                 age_group_id=age_group_ids,
                 sex_id=sex_ids,
                 year_id=2019,
                 measure_id=3,
                 decomp_step='step5',
                 gbd_round_id=6)
ira_ylds = ira_ylds.groupby(['location_id','sex_id','age_group_id'], as_index=False).sum()

pop = get_population(location_id=location_ids,
                    age_group_id=age_group_ids,
                    sex_id=sex_ids,
                    gbd_round_id=6,
                    year_id=2019,
                    decomp_step='step4')

ira_ylds = ira_ylds.merge(pop, on=['location_id','sex_id','age_group_id'])
for i in list(range(0,1000)):
    ira_ylds[f'draw_{i}'] = ira_ylds[f'draw_{i}'] * ira_ylds['population']
ira_ylds = ira_ylds.groupby(['location_id']).sum()
for i in list(range(0,1000)):
    ira_ylds[f'draw_{i}'] = ira_ylds[f'draw_{i}'] / ira_ylds['population'] * 100_000
ira_ylds = ira_ylds.drop(columns=[c for c in ira_ylds.columns if 'draw' not in c])
ira_ylds.head()

,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,draw_106,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
location_id,,,,,,,,,,,,,,,,,,,,,
6,214.225102,216.657461,151.852819,208.604431,183.061954,207.157717,199.245170,198.846079,285.407715,140.671048,...,196.580260,211.505134,276.184178,200.092045,215.164461,184.695145,107.389851,194.660102,164.080221,202.762109
11,809.742321,682.380657,540.657010,713.507046,535.650523,614.637011,691.887717,527.329779,740.871339,464.686349,...,457.670248,691.152648,581.092441,569.453589,621.514402,653.395784,355.674448,468.554738,437.401744,573.793518
15,1165.677602,1024.724016,798.548094,1125.979673,850.254048,1036.408959,1026.833363,851.020927,1114.407576,700.764019,...,860.833218,1035.594794,1032.030641,856.656757,916.625132,1076.488273,511.788271,728.085922,769.820640,904.507905
20,367.290472,368.594553,306.299292,333.001772,305.650688,294.254884,303.917782,253.653451,374.208848,251.087895,...,289.525941,376.053908,257.793100,244.783757,318.369979,370.470222,140.774655,284.897535,239.583945,223.643206
141,481.066950,459.564466,387.321640,477.958164,370.310782,341.843404,392.556730,400.573456,430.104630,313.780486,...,369.308420,400.602578,426.438752,359.536058,344.957734,453.363987,179.689614,276.160709,281.283009,327.360485


In [13]:
ira_prev = get_draws('sequela_id', iron_responsive_anemia_sequelae, 
                 source='como',
                 location_id=location_ids, 
                 age_group_id=age_group_ids,
                 sex_id=sex_ids,
                 year_id=2019,
                 measure_id=5,
                 decomp_step='step5',
                 gbd_round_id=6)
ira_prev = ira_prev.groupby(['location_id','sex_id','age_group_id'], as_index=False).sum()

In [16]:
anemia_diff = calculate_counterfactual_anemia_prevalence(location_ids, sex_ids, age_group_ids,
                                              anemia)
ira_diff = calculate_counterfactual_iron_responsive_anemia_prevalence(ira_prev,
                                                              averted_anemia,
                                                              location_ids,
                                                              sex_ids,
                                                              age_group_ids)
anemia_diff['measure'] = 'anemia'
ira_diff['measure'] = 'iron_responsive_anemia'
deficiency_difference = pd.concat([anemia_diff.reset_index(), ira_diff.reset_index()], ignore_index=True, sort=True)
deficiency_difference.to_pickle('results_raw/iron_deficiency_difference_wra.pkl')
deficiency_difference.head()

,coverage_level,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,...,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure,vehicle,year
0,0.2,9.587623,10.275525,9.041598,10.152091,10.311621,9.835195,9.447689,10.847484,11.527649,...,10.622903,9.006447,10.020565,11.001366,10.005428,11.631691,6,anemia,wheat flour,2025.0
1,0.5,8.937889,10.046273,8.770456,9.923719,9.617997,9.238364,9.329306,10.166411,11.313932,...,10.436533,8.557459,9.243855,10.776112,9.653232,11.319904,6,anemia,wheat flour,2025.0
2,0.8,8.326901,9.821445,8.506575,9.699746,8.964244,8.673498,9.212198,9.521688,11.103588,...,10.252920,8.128637,8.520460,10.554730,9.311753,11.015233,6,anemia,wheat flour,2025.0
3,0.2,37.620093,37.796896,35.708678,39.335924,38.427348,39.331846,36.151862,39.019775,37.972518,...,36.708816,39.286977,38.118841,36.245759,37.406135,41.621380,15,anemia,wheat flour,2025.0
4,0.5,37.478080,37.729522,35.649895,39.288240,38.314434,39.241585,36.131252,38.914741,37.939187,...,36.653011,39.209404,38.001155,36.188651,37.359742,41.568090,15,anemia,wheat flour,2025.0


In [20]:
# check iron responsive portion
impairment_anemia = get_draws(gbd_id_type='rei_id',
         gbd_id=[192],
         source='como',
         location_id=location_ids,
         sex_id=sex_ids,
         age_group_id=age_group_ids,
         gbd_round_id=6,
         year_id=2019,
         decomp_step='step5',
         measure_id=[3])
impairment_anemia = impairment_anemia.loc[impairment_anemia.cause_id==294]
impairment_anemia = impairment_anemia.merge(pop, on=['location_id','sex_id'])
for i in list(range(0,1000)):
    impairment_anemia[f'draw_{i}'] = impairment_anemia[f'draw_{i}'] * impairment_anemia['population']
impairment_anemia = impairment_anemia.groupby(['location_id','measure_id','rei_id']).sum()
for i in list(range(0,1000)):
    impairment_anemia[f'draw_{i}'] = impairment_anemia[f'draw_{i}'] / impairment_anemia['population'] * 100_000
impairment_anemia = impairment_anemia.drop(columns=[c for c in impairment_anemia.columns if 'draw' not in c])
(ira_ylds / impairment_anemia).mean(axis=1)

location_id  measure_id  rei_id
6            3           192       0.954015
11           3           192       0.961034
15           3           192       0.916792
20           3           192       1.078846
141          3           192       0.890837
161          3           192       0.796116
163          3           192       0.799578
164          3           192       0.940607
165          3           192       0.857264
168          3           192       0.682361
171          3           192       0.582179
179          3           192       0.828381
180          3           192       0.608740
181          3           192       0.718089
184          3           192       0.530953
189          3           192       0.677578
190          3           192       0.485159
196          3           192       0.705113
201          3           192       0.572733
202          3           192       0.623709
205          3           192       0.531386
207          3           192       0.587540


In [39]:
anemia_pifs = anemia_rates_averted / (ira_ylds) * 100
anemia_pifs.head()

draw                                              draw_0    draw_1   draw_10  \
location_id vehicle     coverage_level year                                    
6           wheat flour 0.2            2025.0   4.406450  1.553778  2.430108   
                        0.5            2025.0  10.625736  3.839243  5.980650   
                        0.8            2025.0  16.403574  6.071472  9.420579   
15          wheat flour 0.2            2025.0   0.325284  0.160176  0.178984   
                        0.5            2025.0   0.811734  0.400083  0.447088   

draw                                           draw_100   draw_101   draw_102  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0  1.570646   5.739993   4.257205   
                        0.5            2025.0  3.880265  13.842692  10.306447   
                        0.8            2025.0  6.135301  21.370355  15.972591   
15          wheat flour 0.2            2025.0  0.110396   0.328329   0.224560   
                        0.5            2025.0  0.275823   0.819643   0.560776   

draw                                           draw_103   draw_104  draw_105  \
location_id vehicle     coverage_level year                                    
6           wheat flour 0.2            2025.0  0.794927   5.584256  1.206610   
                        0.5            2025.0  1.974236  13.501702  2.987506   
                        0.8            2025.0  3.138016  20.896303  4.734105   
15          wheat flour 0.2            2025.0  0.050025   0.311763  0.074521   
                        0.5            2025.0  0.125028   0.778389  0.186223   

draw                                            draw_106  ...   draw_990  \
location_id vehicle     coverage_level year               ...              
6           wheat flour 0.2            2025.0   6.161111  ...   2.891653   
                        0.5            2025.0  14.834030  ...   7.098357   
                        0.8            2025.0  22.865742  ...  11.152600   
15          wheat flour 0.2            2025.0   0.464779  ...   0.202449   
                        0.5            2025.0   1.159770  ...   0.505661   

draw                                            draw_991   draw_992  \
location_id vehicle     coverage_level year                           
6           wheat flour 0.2            2025.0   2.652263   5.766928   
                        0.5            2025.0   6.492161  13.843256   
                        0.8            2025.0  10.171430  21.272764   
15          wheat flour 0.2            2025.0   0.158434   0.281458   
                        0.5            2025.0   0.395753   0.702629   

draw                                            draw_993  draw_994   draw_995  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0   6.440264  1.317642   3.410360   
                        0.5            2025.0  15.440000  3.263873   8.305919   
                        0.8            2025.0  23.698569  5.174342  12.948796   
15          wheat flour 0.2            2025.0   0.450442  0.154505   0.182388   
                        0.5            2025.0   1.123592  0.385972   0.455524   

draw                                            draw_996  draw_997   draw_998  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0  11.061216  1.807922   3.197209   
                        0.5            2025.0  26.537803  4.471466   7.846075   
                        0.8            2025.0  40.763233  7.077952  12.323709   
15          wheat flour 0.2            2025.0   0.578415  0.186474   0.151769   
                        0.5            2025.0   1.443861  0.465823   0.379189   

draw                                           draw_999  
location_id vehicle     coverage_level year              
6           wheat flour 0.2

In [40]:
anemia_pifs['measure'] = 'pif'
anemia_rates_averted['measure'] = 'rates_averted'
anemia_counts_averted['measure'] = 'counts_averted'

anemia_wra_results = pd.concat([anemia_pifs, anemia_rates_averted, anemia_counts_averted], sort=True).reset_index()
anemia_wra_results = anemia_wra_results.set_index([c for c in anemia_wra_results if 'draw' not in c])

anemia_wra_results.to_pickle('results_raw/iron_anemia_wra.pkl')

anemia_wra_results.head()

draw                                                      draw_0    draw_1  \
location_id vehicle     coverage_level year   measure                        
6           wheat flour 0.2            2025.0 pif       4.406450  1.553778   
                        0.5            2025.0 pif      10.625736  3.839243   
                        0.8            2025.0 pif      16.403574  6.071472   
15          wheat flour 0.2            2025.0 pif       0.325284  0.160176   
                        0.5            2025.0 pif       0.811734  0.400083   

draw                                                    draw_10  draw_100  \
location_id vehicle     coverage_level year   measure                       
6           wheat flour 0.2            2025.0 pif      2.430108  1.570646   
                        0.5            2025.0 pif      5.980650  3.880265   
                        0.8            2025.0 pif      9.420579  6.135301   
15          wheat flour 0.2            2025.0 pif      0.178984  0.110396   
                        0.5            2025.0 pif      0.447088  0.275823   

draw                                                    draw_101   draw_102  \
location_id vehicle     coverage_level year   measure                         
6           wheat flour 0.2            2025.0 pif       5.739993   4.257205   
                        0.5            2025.0 pif      13.842692  10.306447   
                        0.8            2025.0 pif      21.370355  15.972591   
15          wheat flour 0.2            2025.0 pif       0.328329   0.224560   
                        0.5            2025.0 pif       0.819643   0.560776   

draw                                                   draw_103   draw_104  \
location_id vehicle     coverage_level year   measure                        
6           wheat flour 0.2            2025.0 pif      0.794927   5.584256   
                        0.5            2025.0 pif      1.974236  13.501702   
                        0.8            2025.0 pif      3.138016  20.896303   
15          wheat flour 0.2            2025.0 pif      0.050025   0.311763   
                        0.5            2025.0 pif      0.125028   0.778389   

draw                                                   draw_105   draw_106  \
location_id vehicle     coverage_level year   measure                        
6           wheat flour 0.2            2025.0 pif      1.206610   6.161111   
                        0.5            2025.0 pif      2.987506  14.834030   
                        0.8            2025.0 pif      4.734105  22.865742   
15          wheat flour 0.2            2025.0 pif      0.074521   0.464779   
                        0.5            2025.0 pif      0.186223   1.159770   

draw                                                   ...   draw_990  \
location_id vehicle     coverage_level year   measure  ...              
6           wheat flour 0.2            2025.0 pif      ...   2.891653   
                        0.5            2025.0 pif      ...   7.098357   
                        0.8            2025.0 pif      ...  11.152600   
15          wheat flour 0.2            2025.0 pif      ...   0.202449   
                        0.5            2025.0 pif      ...   0.505661   

draw                                                    draw_991   draw_992  \
location_id vehicle     coverage_level year   measure                         
6           wheat flour 0.2            2025.0 pif       2.652263   5.766928   
                        0.5            2025.0 pif       6.492161  13.843256   
                        0.8            2025.0 pif      10.171430  21.272764   
15          wheat flour 0.2            2025.0 pif       0.158434   0.281458   
                        0.5            2025.0 pif       0.395753   0.702629   

draw                                                    draw_993  draw_994  \
location_id vehicle     coverage_level year   measure                        
6           wheat flour 0.2            2025

# MATERNAL DISORDERS

In [21]:
rr = get_draws('rei_id',
                 95,
                 source='rr',
                 location_id=location_ids,
                 age_group_id=age_group_ids,
                 sex_id=sex_ids,
                 year_id=2019,
                 gbd_round_id=6,
                 decomp_step='step4'
                 )
rr = rr.loc[rr.cause_id==367].set_index(['sex_id','age_group_id'])
rr = rr.drop(columns=[c for c in rr.columns if 'draw' not in c])
rr

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v108/lib/python3.7/site-packages/get_draws/sources/rr.py:256: UserWarning: RRs draws returned for rei_id = 95 have location_id(s) = 1 where 179, 163, 214, 161, 165, 190, 189, 196, 201, 180, 15, 164, 20, 202, 205, 171, 11, 184, 168, 6, 141, 207, 181, 213, 522 were requested.
  ", ".join(str(hf) for hf in hdf_filters['location_id'])


draw_0    draw_1    draw_2    draw_3    draw_4  \
sex_id age_group_id                                                     
2      8             1.212408  1.350076  1.086644  1.219068  1.261495   
       13            1.212408  1.350076  1.086644  1.219068  1.261495   
       11            1.212408  1.350076  1.086644  1.219068  1.261495   
       10            1.212408  1.350076  1.086644  1.219068  1.261495   
       9             1.212408  1.350076  1.086644  1.219068  1.261495   
       12            1.212408  1.350076  1.086644  1.219068  1.261495   
       14            1.212408  1.350076  1.086644  1.219068  1.261495   
       7             1.212408  1.350076  1.086644  1.219068  1.261495   
       15            1.212408  1.350076  1.086644  1.219068  1.261495   

                       draw_5    draw_6    draw_7   draw_8    draw_9  ...  \
sex_id age_group_id                                                   ...   
2      8             1.286287  1.092801  1.212885  1.19206  1.194871  ...   
       13            1.286287  1.092801  1.212885  1.19206  1.194871  ...   
       11            1.286287  1.092801  1.212885  1.19206  1.194871  ...   
       10            1.286287  1.092801  1.212885  1.19206  1.194871  ...   
       9             1.286287  1.092801  1.212885  1.19206  1.194871  ...   
       12            1.286287  1.092801  1.212885  1.19206  1.194871  ...   
       14            1.286287  1.092801  1.212885  1.19206  1.194871  ...   
       7             1.286287  1.092801  1.212885  1.19206  1.194871  ...   
       15            1.286287  1.092801  1.212885  1.19206  1.194871  ...   

                     draw_990  draw_991  draw_992  draw_993  draw_994  \
sex_id age_group_id                                                     
2      8             1.189681  1.315712   1.28604  1.257695  1.216908   
       13            1.189681  1.315712   1.28604  1.257695  1.216908   
       11            1.189681  1.315712   1.28604  1.257695  1.216908   
       10            1.189681  1.315712   1.28604  1.257695  1.216908   
       9             1.189681  1.315712   1.28604  1.257695  1.216908   
       12            1.189681  1.315712   1.28604  1.257695  1.216908   
       14            1.189681  1.315712   1.28604  1.257695  1.216908   
       7             1.189681  1.315712   1.28604  1.257695  1.216908   
       15            1.189681  1.315712   1.28604  1.257695  1.216908   

                     draw_995  draw_996  draw_997  draw_998  draw_999  
sex_id age_group_id                                                    
2      8             1.177559  1.315347  1.221176  1.276526  1.243538  
       13            1.177559  1.315347  1.221176  1.276526  1.243538  
       11            1.177559  1.315347  1.221176  1.276526  1.243538  
       10            1.177559  1.315347  1.221176  1.276526  1.243538  
       9             1.177559  1.315347  1.221176  1.276526  1.243538  
       12            1.177559  1.315347  1.221176  1.276526  1.243538  
       14            1.177559  1.315347  1.221176  1.276526  1.243538  
       7             1.177559  1.315347  1.221176  1.276526  1.243538  
       15            1.177559  1.315347  1.221176  1.276526  1.243538  

[9 rows x 1000 columns]

In [22]:
baseline_hgb = get_draws('modelable_entity_id',
                    10487,
                    source='epi',
                    location_id=location_ids,
                    age_group_id=age_group_ids,
                    sex_id=sex_ids,
                    year_id=2019,
                    gbd_round_id=6,
                    decomp_step='step4',
                    status='best')
baseline_hgb = baseline_hgb.set_index(['location_id','sex_id','age_group_id'])
baseline_hgb = baseline_hgb.drop(columns=[c for c in baseline_hgb.columns if 'draw' not in c])
baseline_hgb.head()

draw_0      draw_1     draw_10  \
location_id sex_id age_group_id                                       
11          2      7             125.720458  125.013067  123.877956   
                   8             123.909047  124.923568  130.778339   
                   9             126.279327  129.756698  127.747059   
                   10            127.370087  123.591184  124.936974   
                   11            124.102796  128.072308  127.937545   

                                   draw_100    draw_101    draw_102  \
location_id sex_id age_group_id                                       
11          2      7             131.046273  126.535151  126.539773   
                   8             125.834611  124.179762  131.425035   
                   9             126.493747  127.871689  126.866193   
                   10            127.437921  127.871399  126.539961   
                   11            125.883403  125.754281  125.336822   

                                   draw_103    draw_104    draw_105  \
location_id sex_id age_group_id                                       
11          2      7             128.910306  130.213900  124.539656   
                   8             130.662542  130.573890  124.279944   
                   9             125.532345  127.220828  127.520039   
                   10            128.011740  129.364461  125.843733   
                   11            131.052587  124.180565  130.917344   

                                   draw_106  ...    draw_990    draw_991  \
location_id sex_id age_group_id              ...                           
11          2      7             126.966239  ...  127.598859  128.639075   
                   8             130.325987  ...  129.610821  126.145738   
                   9             125.757247  ...  128.830036  129.892692   
                   10            129.093740  ...  127.874589  126.383536   
                   11            124.230008  ...  125.236726  126.314923   

                                   draw_992    draw_993    draw_994  \
location_id sex_id age_group_id                                       
11          2      7             127.495184  126.026349  126.802325   
                   8             131.445379  125.873082  128.667313   
                   9             129.194790  128.552509  128.126533   
                   10            125.760543  127.463698  128.855356   
                   11            126.743300  127.318785  133.116473   

                                   draw_995    draw_996    draw_997  \
location_id sex_id age_group_id                                       
11          2      7             123.832785  125.791511  126.368115   
                   8             125.066203  132.511309  127.834327   
                   9             125.604651  131.079342  131.531884   
                   10            130.627663  122.278558  124.942197   
                   11            129.348699  126.391037  126.928617   

                                   draw_998    draw_999  
location_id sex_id age_group_id                          
11          2      7             128.945972  125.442573  
                   8             131.196175  130.157132  
                   9             126.830442  130.693367  
                   10            130.590653  132.024848  
                   11            127.330763  124.853824  

[5 rows x 1000 columns]

In [23]:
hgb_diff = (((mean_hgb - baseline_hgb) / 10)
            .reset_index()
            .set_index(['sex_id','age_group_id','location_id','coverage_level','year','vehicle']))
hgb_diff.head()

draw_0  \
sex_id age_group_id location_id coverage_level year   vehicle                 
2      7            6           baseline       NaN    wheat flour  0.000000   
                                                      maize flour  0.000000   
                                0.2            2025.0 wheat flour  0.040954   
                                0.5            2025.0 wheat flour  0.102386   
                                0.8            2025.0 wheat flour  0.163818   

                                                                     draw_1  \
sex_id age_group_id location_id coverage_level year   vehicle                 
2      7            6           baseline       NaN    wheat flour  0.000000   
                                                      maize flour  0.000000   
                                0.2            2025.0 wheat flour  0.013758   
                                0.5            2025.0 wheat flour  0.034395   
                                0.8            2025.0 wheat flour  0.055032   

                                                                    draw_10  \
sex_id age_group_id location_id coverage_level year   vehicle                 
2      7            6           baseline       NaN    wheat flour  0.000000   
                                                      maize flour  0.000000   
                                0.2            2025.0 wheat flour  0.017176   
                                0.5            2025.0 wheat flour  0.042939   
                                0.8            2025.0 wheat flour  0.068703   

                                                                   draw_100  \
sex_id age_group_id location_id coverage_level year   vehicle                 
2      7            6           baseline       NaN    wheat flour  0.000000   
                                                      maize flour  0.000000   
                                0.2            2025.0 wheat flour  0.013541   
                                0.5            2025.0 wheat flour  0.033852   
                                0.8            2025.0 wheat flour  0.054164   

                                                                   draw_101  \
sex_id age_group_id location_id coverage_level year   vehicle                 
2      7            6           baseline       NaN    wheat flour  0.000000   
                                                      maize flour  0.000000   
                                0.2            2025.0 wheat flour  0.042119   
                                0.5            2025.0 wheat flour  0.105296   
                                0.8            2025.0 wheat flour  0.168474   

                                                                   draw_102  \
sex_id age_group_id location_id coverage_level year   vehicle                 
2      7            6           baseline       NaN    wheat flour  0.000000   
                                                      maize flour  0.000000   
                                0.2            2025.0 wheat flour  0.037293   
                                0.5            2025.0 wheat flour  0.093232   
                                0.8            2025.0 wheat flour  0.149172   

                                                                   draw_103  \
sex_id age_group_id location_id coverage_level year   vehicle                 
2      7            6           baseline       NaN    wheat flour  0.000000   
                                                      maize flour  0.000000   
                                0.2            2025.0 wheat flour  0.007086   
                                0.5            2025.0 wheat flour  0.017714   
                                0.8            2025.0 wheat flour  0.028343   

                                                                   draw_104  \
sex_id age_group_id location_id coverage_level year   vehicle                 
2      7            6           b

In [24]:
import math
frac_reduction = ((1 / math.e ** (np.log(rr) * hgb_diff))
                  .reset_index()
                  .set_index(['location_id','sex_id','age_group_id','coverage_level','year','vehicle']))
frac_reduction.head()

draw_0  \
location_id sex_id age_group_id coverage_level year   vehicle                 
6           2      7            baseline       NaN    wheat flour  1.000000   
                                                      maize flour  1.000000   
                                0.2            2025.0 wheat flour  0.992143   
                                0.5            2025.0 wheat flour  0.980473   
                                0.8            2025.0 wheat flour  0.968940   

                                                                     draw_1  \
location_id sex_id age_group_id coverage_level year   vehicle                 
6           2      7            baseline       NaN    wheat flour  1.000000   
                                                      maize flour  1.000000   
                                0.2            2025.0 wheat flour  0.995879   
                                0.5            2025.0 wheat flour  0.989729   
                                0.8            2025.0 wheat flour  0.983617   

                                                                    draw_10  \
location_id sex_id age_group_id coverage_level year   vehicle                 
6           2      7            baseline       NaN    wheat flour  1.000000   
                                                      maize flour  1.000000   
                                0.2            2025.0 wheat flour  0.995056   
                                0.5            2025.0 wheat flour  0.987686   
                                0.8            2025.0 wheat flour  0.980370   

                                                                   draw_100  \
location_id sex_id age_group_id coverage_level year   vehicle                 
6           2      7            baseline       NaN    wheat flour  1.000000   
                                                      maize flour  1.000000   
                                0.2            2025.0 wheat flour  0.995862   
                                0.5            2025.0 wheat flour  0.989688   
                                0.8            2025.0 wheat flour  0.983552   

                                                                   draw_101  \
location_id sex_id age_group_id coverage_level year   vehicle                 
6           2      7            baseline       NaN    wheat flour  1.000000   
                                                      maize flour  1.000000   
                                0.2            2025.0 wheat flour  0.992451   
                                0.5            2025.0 wheat flour  0.981235   
                                0.8            2025.0 wheat flour  0.970146   

                                                                   draw_102  \
location_id sex_id age_group_id coverage_level year   vehicle                 
6           2      7            baseline       NaN    wheat flour  1.000000   
                                                      maize flour  1.000000   
                                0.2            2025.0 wheat flour  0.988834   
                                0.5            2025.0 wheat flour  0.972317   
                                0.8            2025.0 wheat flour  0.956077   

                                                                   draw_103  \
location_id sex_id age_group_id coverage_level year   vehicle                 
6           2      7            baseline       NaN    wheat flour  1.000000   
                                                      maize flour  1.000000   
                                0.2            2025.0 wheat flour  0.998533   
                                0.5            2025.0 wheat flour  0.996337   
                                0.8            2025.0 wheat flour  0.994146   

                                                                   draw_104  \
location_id sex_id age_group_id coverage_level year   vehicle                 
6           2      7            b

In [25]:
maternal_ylds = get_draws('cause_id',
                367,
                source='como',
                location_id=location_ids,
                age_group_id=age_group_ids,
                measure_id=3, #ylds
                metric_id=3, #rate
                sex_id=sex_ids,
                year_id=2019,
                gbd_round_id=6,
                decomp_step='step5')
pop = get_population(location_id=location_ids,
                    sex_id=sex_ids,
                    age_group_id=age_group_ids,
                    gbd_round_id=6,
                    decomp_step='step4')
maternal_ylds = maternal_ylds.merge(pop, on=['location_id','sex_id','age_group_id'])
for i in list(range(0,1000)):
    maternal_ylds[f'draw_{i}'] = maternal_ylds[f'draw_{i}'] * maternal_ylds['population']

maternal_ylds = maternal_ylds.set_index(['location_id','sex_id','age_group_id'])
maternal_ylds = maternal_ylds.drop(columns=[c for c in maternal_ylds.columns if 'draw' not in c])
maternal_ylds.head()

draw_0       draw_1      draw_10  \
location_id sex_id age_group_id                                          
11          2      7                3.586432     4.998050     3.445164   
                   8              585.737491   585.426156   491.528819   
                   9             1922.505580  1276.277180  1917.771792   
                   10            4060.989508  4025.468974  4192.070797   
                   11            4455.327358  3118.082544  3211.384518   

                                    draw_100     draw_101     draw_102  \
location_id sex_id age_group_id                                          
11          2      7                2.116180     2.344429     1.962343   
                   8              528.633002   452.269796   430.703978   
                   9             1740.962970  2239.173970  1613.572678   
                   10            4131.366644  4668.040857  4016.559428   
                   11            4115.780991  3081.222415  3658.435750   

                                    draw_103     draw_104     draw_105  \
location_id sex_id age_group_id                                          
11          2      7                2.024419     2.148836     5.864235   
                   8              459.285431   491.153859   760.852711   
                   9             2040.319616  1924.962518  1333.200821   
                   10            4433.866685  3314.553836  4079.742520   
                   11            2917.687330  2962.917146  3668.764115   

                                    draw_106  ...     draw_990     draw_991  \
location_id sex_id age_group_id               ...                             
11          2      7                1.384865  ...     3.034912     4.170687   
                   8              271.857649  ...   494.872637  1021.817513   
                   9             1161.819940  ...  1439.148492  3564.418236   
                   10            2832.033212  ...  3605.415130  3914.302468   
                   11            2678.979705  ...  2823.482936  2935.157893   

                                    draw_992     draw_993     draw_994  \
location_id sex_id age_group_id                                          
11          2      7                3.349819     2.861947     2.151309   
                   8              460.191849   593.479878   463.517862   
                   9             2162.130717  1910.494790  2328.013999   
                   10            3547.129545  2637.807829  3262.049063   
                   11            3311.509870  2782.787451  3235.297050   

                                    draw_995     draw_996     draw_997  \
location_id sex_id age_group_id                                          
11          2      7                3.770873     1.982540     2.928532   
                   8              621.126836   189.985218   618.708213   
                   9             1893.261671  1009.037569  1664.215066   
                   10            3877.918388  2138.601526  2846.766586   
                   11            4177.115755  1481.247044  2691.780737   

                                    draw_998     draw_999  
location_id sex_id age_group_id                            
11          2      7                2.121362     2.643029  
                   8              301.103977   302.589682  
                   9             1180.678435  1297.549184  
                   10            2840.064448  2267.987208  
                   11            3238.916812  3085.059741  

[5 rows x 1000 columns]

In [26]:
maternal_ylls = get_draws('cause_id',
                367,
                source='codcorrect',
                location_id=location_ids,
                age_group_id=age_group_ids,
                measure_id=4,
                metric_id=1,
                sex_id=sex_ids,
                year_id=2019,
                gbd_round_id=6,
                decomp_step='step5',
                status='latest')

maternal_ylls = maternal_ylls.set_index(['location_id','sex_id','age_group_id'])
maternal_ylls = maternal_ylls.drop(columns=[c for c in maternal_ylls.columns if 'draw' not in c])
maternal_ylls.head()

draw_0        draw_1       draw_10  \
location_id sex_id age_group_id                                             
11          2      7               675.402102    576.379372    851.124053   
                   8             17035.059448  10267.173550  11733.353193   
                   9             17106.940174  21064.130315  27093.440891   
                   10            27529.648730  18090.108355  25961.112723   
                   11            19693.763706  18457.835506  21981.038867   

                                     draw_100      draw_101      draw_102  \
location_id sex_id age_group_id                                             
11          2      7               486.538016    889.418333    551.659538   
                   8             12122.816532  17403.603694  12930.613203   
                   9             21927.797991  17009.881472  23397.324135   
                   10            19800.344100  20680.135052  22574.962807   
                   11            20389.021204  18302.948716  16748.808596   

                                     draw_103      draw_104      draw_105  \
location_id sex_id age_group_id                                             
11          2      7               615.710799    627.343427    624.991833   
                   8             14887.891892  10069.628672  12324.197887   
                   9             21598.236991  17174.517221  17982.661058   
                   10            15412.396600  11020.159270  23203.427202   
                   11            19062.819906  12802.212035  20691.140082   

                                     draw_106  ...      draw_990  \
location_id sex_id age_group_id                ...                 
11          2      7               691.792727  ...    380.705751   
                   8             18833.933931  ...  11551.797249   
                   9             22898.693773  ...  15732.960230   
                   10            18208.801935  ...  14247.158825   
                   11            27108.162649  ...  15675.043762   

                                     draw_991      draw_992      draw_993  \
location_id sex_id age_group_id                                             
11          2      7               708.622357    832.972619    557.511990   
                   8             13423.341485  15491.393755  13212.484313   
                   9             23153.955959  23991.776401  21178.278535   
                   10            20366.550781  23152.770207  15508.642945   
                   11            24834.778001  36310.402370  17662.119971   

                                     draw_994      draw_995      draw_996  \
location_id sex_id age_group_id                                             
11          2      7               770.748074    872.983215    620.485120   
                   8             15080.320312  13475.627776  14279.332341   
                   9             24916.795059  16307.667557  23091.716613   
                   10            36355.918759  19408.068056  30866.682918   
                   11            33861.664309  27735.686668  29317.248900   

                                     draw_997      draw_998      draw_999  
location_id sex_id age_group_id                                            
11          2      7               782.854251    747.291351    695.196266  
                   8             12581.398051  14248.173475  15389.637181  
                   9             27949.115225  24137.241881  28621.722688  
                   10            19005.360367  33206.852789  26453.907884  
                   11            26024.570876  28912.464753  37564.705147  

[5 rows x 1000 columns]

In [27]:
maternal_dalys = maternal_ylds + maternal_ylls
maternal_dalys.head()

draw_0       draw_1      draw_10  \
location_id sex_id age_group_id                                          
6           2      7               44.902678    56.457266    47.074367   
                   8              990.623002  1199.652620   986.104327   
                   9             4250.921183  4239.550767  4720.296626   
                   10            8883.877922  7007.399587  6740.677688   
                   11            7440.586870  7313.477640  7131.470259   

                                    draw_100      draw_101     draw_102  \
location_id sex_id age_group_id                                           
6           2      7               52.292281     38.002272    54.049400   
                   8             1014.225848   1031.754124  1109.396091   
                   9             4175.200050   5585.117819  4258.492862   
                   10            9814.039185  11528.808303  6841.358463   
                   11            9080.354529   7564.573551  7042.326436   

                                    draw_103     draw_104     draw_105  \
location_id sex_id age_group_id                                          
6           2      7               60.600345    42.537419    42.180757   
                   8             1030.896982  1016.810159   832.292742   
                   9             4351.610678  3617.110619  3489.653089   
                   10            9253.426442  9308.550115  8324.751786   
                   11            7644.317524  7798.476480  6619.792520   

                                    draw_106  ...     draw_990     draw_991  \
location_id sex_id age_group_id               ...                             
6           2      7               55.958593  ...    50.382559    53.742423   
                   8              844.502166  ...  1121.527845  1261.081593   
                   9             3066.856388  ...  4073.540341  6093.246205   
                   10            6273.413696  ...  8171.185779  9992.402914   
                   11            4963.253186  ...  7246.503120  7134.547658   

                                     draw_992     draw_993     draw_994  \
location_id sex_id age_group_id                                           
6           2      7                43.164737    60.383373    54.043236   
                   8              1136.994702  1185.964400  1191.772056   
                   9              4952.530456  4775.817120  5499.250261   
                   10            10304.765964  8704.692371  9173.562820   
                   11             6851.602339  7771.890004  9520.737173   

                                    draw_995     draw_996     draw_997  \
location_id sex_id age_group_id                                          
6           2      7               50.058619    54.281501    49.366342   
                   8             1148.911602   816.892052   997.743811   
                   9             4290.731225  3282.617123  5310.974419   
                   10            8601.487819  6702.282379  7557.843653   
                   11            7727.499022  5099.537199  7454.614608   

                                    draw_998     draw_999  
location_id sex_id age_group_id                            
6           2      7               45.348378    49.982552  
                   8              855.981620  1134.372198  
                   9             3566.179770  3751.852506  
                   10            8685.246028  8899.861467  
                   11            7573.202847  8345.598207  

[5 rows x 1000 columns]

In [28]:
counterfactual_maternal_dalys = maternal_dalys * frac_reduction
counterfactual_maternal_dalys = (counterfactual_maternal_dalys.groupby(['location_id','vehicle','coverage_level','year']).sum())
maternal_counts_averted = maternal_dalys.groupby('location_id').sum() - counterfactual_maternal_dalys
maternal_counts_averted.head()

draw_0      draw_1  \
location_id vehicle     coverage_level year                             
6           wheat flour 0.2            2025.0  246.043224  114.648905   
                        0.5            2025.0  611.488066  285.736969   
                        0.8            2025.0  972.634349  455.768512   
15          wheat flour 0.2            2025.0   25.138229   13.339861   
                        0.5            2025.0   62.808570   33.336906   

                                                  draw_10    draw_100  \
location_id vehicle     coverage_level year                             
6           wheat flour 0.2            2025.0  133.595128  135.481110   
                        0.5            2025.0  332.750395  337.652419   
                        0.8            2025.0  530.430529  538.570248   
15          wheat flour 0.2            2025.0   13.170119   10.549573   
                        0.5            2025.0   32.913905   26.366376   

                                                 draw_101     draw_102  \
location_id vehicle     coverage_level year                              
6           wheat flour 0.2            2025.0  246.396030   301.100822   
                        0.5            2025.0  612.507096   746.459621   
                        0.8            2025.0  974.480586  1184.379657   
15          wheat flour 0.2            2025.0   16.691475    16.256175   
                        0.5            2025.0   41.712433    40.618897   

                                                 draw_103    draw_104  \
location_id vehicle     coverage_level year                             
6           wheat flour 0.2            2025.0   44.018761  200.130668   
                        0.5            2025.0  109.925878  497.726238   
                        0.8            2025.0  175.688047  792.230114   
15          wheat flour 0.2            2025.0    2.602749   14.055453   
                        0.5            2025.0    6.506301   35.126883   

                                                 draw_105     draw_106  ...  \
location_id vehicle     coverage_level year                             ...   
6           wheat flour 0.2            2025.0   66.804526   256.188094  ...   
                        0.5            2025.0  166.698167   634.639860  ...   
                        0.8            2025.0  266.217285  1006.208344  ...   
15          wheat flour 0.2            2025.0    4.199926    23.856820  ...   
                        0.5            2025.0   10.498449    59.592525  ...   

                                                 draw_990    draw_991  \
location_id vehicle     coverage_level year                             
6           wheat flour 0.2            2025.0  112.760892  219.088243   
                        0.5            2025.0  281.094797  545.013246   
                        0.8            2025.0  448.464712  867.717956   
15          wheat flour 0.2            2025.0    7.673631   12.134801   
                        0.5            2025.0   19.179407   30.326339   

                                                  draw_992     draw_993  \
location_id vehicle     coverage_level year                               
6           wheat flour 0.2            2025.0   445.346687   359.147403   
                        0.5            2025.0  1102.038895   890.005606   
                        0.8            2025.0  1745.383451  1411.575153   
15          wheat flour 0.2            2025.0    23.246033    28.638273   
                        0.5            2025.0    58.082885    71.543190   

                                                 draw_994    draw_995  \
location_id vehicle     coverage_level year                             
6           wheat flour 0.2            2025.0   74.237836  149.346438   
                        0.5            2025.0  185.288835  372.038893   
                        0.8            2025.0  295.974002  593.148768   
15          wheat flour 0.2            

In [29]:
maternal_rates_averted = (maternal_counts_averted.reset_index()
                 .merge(pop.groupby('location_id', as_index=False).sum().drop(columns='year_id'), 
                        on='location_id'))
for i in list(range(0,1000)):
    maternal_rates_averted[f'draw_{i}'] = (maternal_rates_averted[f'draw_{i}'] / maternal_rates_averted['population']) * 100_000
maternal_rates_averted = maternal_rates_averted.set_index(['location_id','vehicle','coverage_level','year'])
maternal_rates_averted = maternal_rates_averted.drop(columns=[c for c in maternal_rates_averted.columns if 'draw' not in c])
maternal_rates_averted.head()

draw_0    draw_1   draw_10  \
location_id vehicle     coverage_level year                                   
6           wheat flour 0.2            2025.0  0.055205  0.025724  0.029975   
                        0.5            2025.0  0.137199  0.064111  0.074659   
                        0.8            2025.0  0.218229  0.102260  0.119012   
15          wheat flour 0.2            2025.0  0.132276  0.070194  0.069301   
                        0.5            2025.0  0.330496  0.175417  0.173192   

                                               draw_100  draw_101  draw_102  \
location_id vehicle     coverage_level year                                   
6           wheat flour 0.2            2025.0  0.030398  0.055284  0.067558   
                        0.5            2025.0  0.075759  0.137428  0.167483   
                        0.8            2025.0  0.120839  0.218644  0.265738   
15          wheat flour 0.2            2025.0  0.055511  0.087830  0.085539   
                        0.5            2025.0  0.138739  0.219489  0.213735   

                                               draw_103  draw_104  draw_105  \
location_id vehicle     coverage_level year                                   
6           wheat flour 0.2            2025.0  0.009876  0.044903  0.014989   
                        0.5            2025.0  0.024664  0.111674  0.037402   
                        0.8            2025.0  0.039419  0.177752  0.059731   
15          wheat flour 0.2            2025.0  0.013696  0.073959  0.022100   
                        0.5            2025.0  0.034236  0.184836  0.055242   

                                               draw_106  ...  draw_990  \
location_id vehicle     coverage_level year              ...             
6           wheat flour 0.2            2025.0  0.057481  ...  0.025300   
                        0.5            2025.0  0.142394  ...  0.063069   
                        0.8            2025.0  0.225762  ...  0.100622   
15          wheat flour 0.2            2025.0  0.125534  ...  0.040378   
                        0.5            2025.0  0.313573  ...  0.100921   

                                               draw_991  draw_992  draw_993  \
location_id vehicle     coverage_level year                                   
6           wheat flour 0.2            2025.0  0.049157  0.099922  0.080582   
                        0.5            2025.0  0.122284  0.247264  0.199690   
                        0.8            2025.0  0.194689  0.391610  0.316714   
15          wheat flour 0.2            2025.0  0.063853  0.122320  0.150693   
                        0.5            2025.0  0.159576  0.305630  0.376457   

                                               draw_994  draw_995  draw_996  \
location_id vehicle     coverage_level year                                   
6           wheat flour 0.2            2025.0  0.016657  0.033509  0.066454   
                        0.5            2025.0  0.041573  0.083474  0.164482   
                        0.8            2025.0  0.066407  0.133084  0.260564   
15          wheat flour 0.2            2025.0  0.048292  0.060931  0.107296   
                        0.5            2025.0  0.120702  0.152291  0.268069   

                                               draw_997  draw_998  draw_999  
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0  0.017872  0.033272  0.027302  
                        0.5            2025.0  0.044591  0.082847  0.068055  
                        0.8            2025.0  0.071203  0.132026  0.108570  
15          wheat flour 0.2            2025.0  0.047558  0.035414  0.031340  
                        0.5            2025.0  0.118868  0.088515  0.078332  

[5 rows x 1000 columns]

In [30]:
iron_attributable_maternal_disorder_burden = get_draws(gbd_id_type=['rei_id','cause_id'],
                                                      gbd_id=[95,367],
                                                      location_id=location_ids,
                                                      sex_id=sex_ids,
                                                      age_group_id=age_group_ids,
                                                      measure_id=2, # dalys
                                                      metric_id=1, # number
                                                      source='burdenator',
                                                      decomp_step='step5',
                                                      gbd_round_id=6,
                                                      year_id=2019)
iron_attributable_maternal_disorder_burden = iron_attributable_maternal_disorder_burden.groupby('location_id').sum()
iron_attributable_maternal_disorder_burden = iron_attributable_maternal_disorder_burden.drop(columns=[c for c in iron_attributable_maternal_disorder_burden.columns if 'draw' not in c])
iron_attributable_maternal_disorder_burden = (iron_attributable_maternal_disorder_burden.reset_index()
                                              .merge(pop.groupby('location_id').sum().reset_index(),
                                                     on='location_id'))
for i in list(range(0,1000)):
    iron_attributable_maternal_disorder_burden[f'draw_{i}'] = iron_attributable_maternal_disorder_burden[f'draw_{i}'] / iron_attributable_maternal_disorder_burden['population']
iron_attributable_maternal_disorder_burden = iron_attributable_maternal_disorder_burden.set_index(['location_id'])
iron_attributable_maternal_disorder_burden = iron_attributable_maternal_disorder_burden.drop(columns=[c for c in iron_attributable_maternal_disorder_burden.columns if 'draw' not in c])

iron_attributable_maternal_disorder_burden

,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,draw_106,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
location_id,,,,,,,,,,,,,,,,,,,,,
6,0.000008,0.000011,0.000009,0.000013,0.000008,0.000010,0.000008,0.000007,0.000008,0.000008,...,0.000007,0.000012,0.000012,0.000010,0.000009,0.000006,0.000008,0.000008,0.000009,0.000009
11,0.000202,0.000254,0.000321,0.000299,0.000165,0.000301,0.000199,0.000119,0.000201,0.000324,...,0.000116,0.000313,0.000296,0.000214,0.000280,0.000169,0.000321,0.000206,0.000293,0.000318
15,0.000273,0.000342,0.000357,0.000381,0.000235,0.000301,0.000198,0.000218,0.000211,0.000272,...,0.000179,0.000307,0.000331,0.000288,0.000253,0.000243,0.000283,0.000248,0.000228,0.000201
20,0.000033,0.000051,0.000045,0.000063,0.000030,0.000061,0.000033,0.000030,0.000035,0.000050,...,0.000022,0.000048,0.000036,0.000036,0.000031,0.000032,0.000029,0.000035,0.000045,0.000035
141,0.000036,0.000056,0.000059,0.000077,0.000048,0.000073,0.000040,0.000042,0.000038,0.000059,...,0.000058,0.000062,0.000071,0.000042,0.000042,0.000052,0.000072,0.000045,0.000069,0.000051
161,0.000384,0.000573,0.000565,0.000497,0.000284,0.000408,0.000340,0.000330,0.000258,0.000510,...,0.000303,0.000573,0.000510,0.000400,0.000408,0.000328,0.000406,0.000440,0.000465,0.000352
163,0.000448,0.000637,0.000630,0.000786,0.000430,0.000694,0.000356,0.000377,0.000546,0.000714,...,0.000386,0.000743,0.000584,0.000452,0.000344,0.000384,0.000705,0.000410,0.000613,0.000568
164,0.000290,0.000480,0.000432,0.000576,0.000325,0.000403,0.000337,0.000300,0.000235,0.000512,...,0.000230,0.000526,0.000513,0.000466,0.000391,0.000281,0.000541,0.000344,0.000445,0.000298
165,0.000927,0.001274,0.001052,0.000971,0.000779,0.001226,0.000869,0.000584,0.000875,0.001168,...,0.000666,0.001024,0.001191,0.000777,0.000744,0.000564,0.001107,0.000652,0.000938,0.001070


In [31]:
maternal_disorders_pif = maternal_rates_averted / (iron_attributable_maternal_disorder_burden * 100_000) * 100
maternal_disorders_pif.head()

draw_0    draw_1    draw_10  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0   6.915258  2.334143   3.288578   
                        0.5            2025.0  17.186403  5.817334   8.190985   
                        0.8            2025.0  27.336732  9.279016  13.057080   
15          wheat flour 0.2            2025.0   0.484211  0.205528   0.193999   
                        0.5            2025.0   1.209814  0.513624   0.484829   

                                               draw_100   draw_101   draw_102  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0  2.304014   6.856136   6.555555   
                        0.5            2025.0  5.742172  17.043425  16.251888   
                        0.8            2025.0  9.159013  27.115582  25.786265   
15          wheat flour 0.2            2025.0  0.145648   0.373847   0.283943   
                        0.5            2025.0  0.364015   0.934255   0.709481   

                                               draw_103   draw_104  draw_105  \
location_id vehicle     coverage_level year                                    
6           wheat flour 0.2            2025.0  1.285333   6.594152  1.955703   
                        0.5            2025.0  3.209798  16.399698  4.880091   
                        0.8            2025.0  5.130031  26.103375  7.793514   
15          wheat flour 0.2            2025.0  0.069276   0.339277  0.104674   
                        0.5            2025.0  0.173175   0.847909  0.261651   

                                                draw_106  ...   draw_990  \
location_id vehicle     coverage_level year               ...              
6           wheat flour 0.2            2025.0   7.128014  ...   3.793817   
                        0.5            2025.0  17.657815  ...   9.457378   
                        0.8            2025.0  27.996100  ...  15.088505   
15          wheat flour 0.2            2025.0   0.462170  ...   0.224961   
                        0.5            2025.0   1.154467  ...   0.562265   

                                                draw_991   draw_992  \
location_id vehicle     coverage_level year                           
6           wheat flour 0.2            2025.0   4.229587   8.476596   
                        0.5            2025.0  10.521701  20.975879   
                        0.8            2025.0  16.751646  33.221107   
15          wheat flour 0.2            2025.0   0.208294   0.369473   
                        0.5            2025.0   0.520552   0.923170   

                                                draw_993  draw_994   draw_995  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0   8.231935  1.838568   5.246620   
                        0.5            2025.0  20.399613  4.588848  13.069925   
                        0.8            2025.0  32.354388  7.330067  20.837633   
15          wheat flour 0.2            2025.0   0.523206  0.190604   0.251007   
                        0.5            2025.0   1.307057  0.476400   0.627363   

                                                draw_996  draw_997   draw_998  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0   8.299158  2.249124   3.541348   
                        0.5            2025.0  20.541599  5.611584   8.817939   
                        0.8            2025.0  32.540858  8.960621  14.052315   
15          wheat flour 0.2            2025.0   0.379719  0.191612   0.155552   
                        0.5            2025.0   0.948697  0.478925   0.388789   

                                                draw_999  
location_id vehicle     coverage_level year               
6           wheat flour 0.2            2025.0   2.884919  
         

In [32]:
maternal_disorders_pif['measure'] = 'pif'
maternal_rates_averted['measure'] = 'rates_averted'
maternal_counts_averted['measure'] = 'counts_averted'

maternal_disorders_wra_results = pd.concat([maternal_disorders_pif, maternal_rates_averted, maternal_counts_averted], sort=True).reset_index()
maternal_disorders_wra_results = maternal_disorders_wra_results.set_index([c for c in maternal_disorders_wra_results if 'draw' not in c])

maternal_disorders_wra_results.to_pickle('results_raw/iron_maternal_disorders_wra.pkl')

maternal_disorders_wra_results.head()

draw_0    draw_1  \
location_id vehicle     coverage_level year   measure                        
6           wheat flour 0.2            2025.0 pif       6.915258  2.334143   
                        0.5            2025.0 pif      17.186403  5.817334   
                        0.8            2025.0 pif      27.336732  9.279016   
15          wheat flour 0.2            2025.0 pif       0.484211  0.205528   
                        0.5            2025.0 pif       1.209814  0.513624   

                                                         draw_10  draw_100  \
location_id vehicle     coverage_level year   measure                        
6           wheat flour 0.2            2025.0 pif       3.288578  2.304014   
                        0.5            2025.0 pif       8.190985  5.742172   
                        0.8            2025.0 pif      13.057080  9.159013   
15          wheat flour 0.2            2025.0 pif       0.193999  0.145648   
                        0.5            2025.0 pif       0.484829  0.364015   

                                                        draw_101   draw_102  \
location_id vehicle     coverage_level year   measure                         
6           wheat flour 0.2            2025.0 pif       6.856136   6.555555   
                        0.5            2025.0 pif      17.043425  16.251888   
                        0.8            2025.0 pif      27.115582  25.786265   
15          wheat flour 0.2            2025.0 pif       0.373847   0.283943   
                        0.5            2025.0 pif       0.934255   0.709481   

                                                       draw_103   draw_104  \
location_id vehicle     coverage_level year   measure                        
6           wheat flour 0.2            2025.0 pif      1.285333   6.594152   
                        0.5            2025.0 pif      3.209798  16.399698   
                        0.8            2025.0 pif      5.130031  26.103375   
15          wheat flour 0.2            2025.0 pif      0.069276   0.339277   
                        0.5            2025.0 pif      0.173175   0.847909   

                                                       draw_105   draw_106  \
location_id vehicle     coverage_level year   measure                        
6           wheat flour 0.2            2025.0 pif      1.955703   7.128014   
                        0.5            2025.0 pif      4.880091  17.657815   
                        0.8            2025.0 pif      7.793514  27.996100   
15          wheat flour 0.2            2025.0 pif      0.104674   0.462170   
                        0.5            2025.0 pif      0.261651   1.154467   

                                                       ...   draw_990  \
location_id vehicle     coverage_level year   measure  ...              
6           wheat flour 0.2            2025.0 pif      ...   3.793817   
                        0.5            2025.0 pif      ...   9.457378   
                        0.8            2025.0 pif      ...  15.088505   
15          wheat flour 0.2            2025.0 pif      ...   0.224961   
                        0.5            2025.0 pif      ...   0.562265   

                                                        draw_991   draw_992  \
location_id vehicle     coverage_level year   measure                         
6           wheat flour 0.2            2025.0 pif       4.229587   8.476596   
                        0.5            2025.0 pif      10.521701  20.975879   
                        0.8            2025.0 pif      16.751646  33.221107   
15          wheat flour 0.2            2025.0 pif       0.208294   0.369473   
                        0.5            2025.0 pif       0.520552   0.923170   

                                                        draw_993  draw_994  \
location_id vehicle     coverage_level year   measure                        
6           wheat flour 0.2            2025.0 pif       8.231935  1.838568   
                

# OVERALL

In [41]:
overall_counts_averted = anemia_counts_averted + maternal_counts_averted
overall_rates_averted = anemia_rates_averted + maternal_rates_averted
overall_counts_averted.head()

draw                                                  draw_0        draw_1  \
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0   42318.298125  15118.378930   
                        0.5            2025.0  102064.740914  37358.578685   
                        0.8            2025.0  157592.000405  59083.658616   
15          wheat flour 0.2            2025.0     745.737170    325.269085   
                        0.5            2025.0    1861.035738    812.466053   

draw                                                draw_10      draw_100  \
location_id vehicle     coverage_level year                                 
6           wheat flour 0.2            2025.0  16580.532535  14738.362902   
                        0.5            2025.0  40809.710493  36413.916750   
                        0.8            2025.0  64288.779548  57580.734841   
15          wheat flour 0.2            2025.0    284.794464    246.779466   
                        0.5            2025.0    711.408489    616.586798   

draw                                                draw_101       draw_102  \
location_id vehicle     coverage_level year                                   
6           wheat flour 0.2            2025.0   47078.756957   39607.411754   
                        0.5            2025.0  113554.454501   95904.766854   
                        0.8            2025.0  175334.308473  148657.577583   
15          wheat flour 0.2            2025.0     547.222322     458.555259   
                        0.5            2025.0    1366.132408    1145.138199   

draw                                               draw_103       draw_104  \
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0   7103.152135   49690.300410   
                        0.5            2025.0  17641.607706  120155.838753   
                        0.8            2025.0  28042.008797  185984.600848   
15          wheat flour 0.2            2025.0    100.223601     518.271555   
                        0.5            2025.0    250.488714    1294.021116   

draw                                               draw_105       draw_106  \
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0  15415.415115   38884.024079   
                        0.5            2025.0  38169.108112   93638.392435   
                        0.8            2025.0  60486.137549  144365.759362   
15          wheat flour 0.2            2025.0    162.025138     642.829039   
                        0.5            2025.0    404.891465    1604.121005   

draw                                           ...      draw_991  \
location_id vehicle     coverage_level year    ...                 
6           wheat flour 0.2            2025.0  ...  25221.053605   
                        0.5            2025.0  ...  61744.365118   
                        0.8            2025.0  ...  96750.261327   
15          wheat flour 0.2            2025.0  ...    323.945746   
                        0.5            2025.0  ...    809.198661   

draw                                                draw_992       draw_993  \
location_id vehicle     coverage_level year                                   
6           wheat flour 0.2            2025.0   71432.531816   57793.281749   
                        0.5            2025.0  171503.638436  138583.597888   
                        0.8            2025.0  263599.457170  212754.905118   
15          wheat flour 0.2            2025.0     575.270501     761.966524   
                        0.5            2025.0    1436.152226    1900.771756   

draw                                               draw_994       draw_995  \
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0  12710.087920   28222.575666   


In [42]:
overall_pif = overall_rates_averted / ((ira_ylds + iron_attributable_maternal_disorder_burden)) * 100
overall_pif.head()

draw_0    draw_1   draw_10  \
location_id vehicle     coverage_level year                                    
6           wheat flour 0.2            2025.0   4.432219  1.565651  2.449847   
                        0.5            2025.0  10.689780  3.868834  6.029815   
                        0.8            2025.0  16.505443  6.118671  9.498952   
15          wheat flour 0.2            2025.0   0.336632  0.167026  0.187663   
                        0.5            2025.0   0.840086  0.417201  0.468776   

                                               draw_100   draw_101   draw_102  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0  1.585218   5.770192   4.289817   
                        0.5            2025.0  3.916582  13.917764  10.387295   
                        0.8            2025.0  6.193228  21.489791  16.100869   
15          wheat flour 0.2            2025.0  0.115326   0.338659   0.232813   
                        0.5            2025.0  0.288145   0.845457   0.581399   

                                               draw_103   draw_104  draw_105  \
location_id vehicle     coverage_level year                                    
6           wheat flour 0.2            2025.0  0.799883   5.606837  1.211861   
                        0.5            2025.0  1.986615  13.557862  3.000611   
                        0.8            2025.0  3.157801  20.985694  4.755033   
15          wheat flour 0.2            2025.0  0.051359   0.320453  0.076504   
                        0.5            2025.0  0.128362   0.800108  0.191180   

                                                draw_106  ...   draw_991  \
location_id vehicle     coverage_level year               ...              
6           wheat flour 0.2            2025.0   6.201973  ...   2.675504   
                        0.5            2025.0  14.935253  ...   6.549977   
                        0.8            2025.0  23.026230  ...  10.263479   
15          wheat flour 0.2            2025.0   0.482693  ...   0.164600   
                        0.5            2025.0   1.204516  ...   0.411162   

                                                draw_992   draw_993  draw_994  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0   5.803108   6.480536  1.325383   
                        0.5            2025.0  13.932784  15.539798  3.283195   
                        0.8            2025.0  21.414557  23.856852  5.205205   
15          wheat flour 0.2            2025.0   0.293310   0.468033  0.159773   
                        0.5            2025.0   0.732243   1.167537  0.399140   

                                                draw_995   draw_996  draw_997  \
location_id vehicle     coverage_level year                                     
6           wheat flour 0.2            2025.0   3.428503  11.123096  1.817103   
                        0.5            2025.0   8.351114  26.690965  4.494373   
                        0.8            2025.0  13.020852  41.005864  7.114530   
15          wheat flour 0.2            2025.0   0.188049   0.599379  0.193006   
                        0.5            2025.0   0.469670   1.496240  0.482149   

                                                draw_998  draw_999  measure  
location_id vehicle     coverage_level year                                  
6           wheat flour 0.2            2025.0   3.217487  2.490457      NaN  
                        0.5            2025.0   7.896566  6.139852      NaN  
                        0.8            2025.0  12.404173  9.687926      NaN  
15          wheat flour 0.2            2025.0   0.156370  0.157282      NaN  
                        0.5            2025.0   0.390687  0.392965      NaN  

[5 rows x 1001 columns]

In [43]:
overall_pif['measure'] = 'pif'
overall_counts_averted['measure'] = 'counts_averted'
overall_rates_averted['measure'] = 'rates_averted'
iron_wra_results = pd.concat([overall_pif.reset_index(),
                         overall_counts_averted.reset_index(),
                         overall_rates_averted.reset_index()],
                        ignore_index=True)
iron_wra_results.to_pickle('results_raw/iron_wra.pkl')